In [1]:
import qlib
from qlib.constant import REG_CN
provider_uri = "../../data/qlib_data/cn_data_ak"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

[18068:MainThread](2022-08-30 20:50:57,760) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[18068:MainThread](2022-08-30 20:50:58,520) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[18068:MainThread](2022-08-30 20:50:58,521) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/PycharmProjects/QlibPlatform/data/qlib_data/cn_data_ak')}


In [8]:
from qlib.contrib.data.handler import Alpha158
from qlib.data.dataset import TSDatasetH
from qlib.contrib.model.pytorch_alstm_ts import ALSTM

# 配置数据
train_period = ("2015-01-01", "2019-12-31")
valid_period = ("2020-01-01", "2020-12-31")
test_period = ("2021-01-01", "2022-03-01")

dh = Alpha158(instruments='sh000300', 
              start_time=train_period[0], 
              end_time=test_period[1],
              infer_processors={}
              
             )
ds = TSDatasetH(handler=dh,
                step_len=40, # 时间步数
                segments={"train": train_period, 
                          "valid": valid_period, 
                          "test": test_period})

Please install necessary libs for CatBoostModel.
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).
ModuleNotFoundError.  PyTorch models are skipped (optional: maybe installing pytorch can fix it).


ModuleNotFoundError: No module named 'torch'